In [1]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any

import torch
import torch.nn as nn
import pandas as pd
import pickle
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from numpy import save
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
folder_ = 'drive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/'

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install sentence_transformers

     |████████████████████████████████| 4.0 MB 9.7 MB/s 
     |████████████████████████████████| 895 kB 66.9 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 6.6 MB 31.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 18.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=aaa16dec3aa781102f1d6ce0dac670767905338c6efeba79b8414da5fbb4e3b6
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [4]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch import Tensor
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig,T5Tokenizer
from transformers import pipeline, AutoTokenizer

In [5]:
train_df=pd.read_csv(folder_ + 'Data/Train.csv')
train_df_english=pd.read_csv(folder_ + 'Data/train_google_translated.csv')

#test_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/test.csv')

In [6]:
train_df['Text']=train_df['Text'].tolist()
train_df_english['Text']=train_df_english['Text'].tolist()
#test_df['Text']=test_df['Text'].tolist()
#test_batch_sentences=[]

#for frame in train_df['Text']:
#    train_batch_sentences.append(frame)
#for sentence in test_df['Text']:
#    test_batch_sentences.append(frame)

In [7]:
print(len(train_df))
print(len(train_df_english))

1436
1436


In [8]:
chichewa_embd = np.load(folder_ + 'Data/' + 'embeddings_chichewa_512_final.npy')

In [9]:
english_embd = np.load(folder_ + 'Data/' + 'embeddings_english_512_final.npy')

In [13]:
final_comb_arr = np.hstack((chichewa_embd,english_embd))

In [14]:
final_comb_arr.shape

(1436, 1024)

In [15]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [16]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [17]:
print(Y_train.shape)
print(Y_train[0])

(1436, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
print('Building model...')
model = Sequential()
model.add(Dense(2048, input_shape=(1024,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
#model.add(Dense(1024, input_shape=(2048,)))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Dense(20))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#make one, takes in, double that amount, english and chichewa

Building model...


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              2099200   
                                                                 
 activation (Activation)     (None, 2048)              0         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                40980     
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                                 
Total params: 2,140,180
Trainable params: 2,140,180
Non-trainable params: 0
_________________________________________________________________


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
        final_comb_arr, Y_train, test_size=0.3, random_state=42, stratify=Y_train)

In [21]:
X_train.shape

(1005, 1024)

In [22]:
X_train[0]

array([-0.02148693, -0.0798831 ,  0.02649599, ...,  0.17925473,
       -0.07263725, -0.07171253], dtype=float32)

In [24]:
X_train[0].shape

(1024,)

In [23]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [25]:
#train_dataset = tf.data.Dataset.from_tensor_slices((values.detach().numpy(), Y_train[:400]))
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [26]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [27]:
result = model.fit(train_dataset,
                    epochs=50, batch_size=32,
                    verbose=1)

Epoch 1/50
32/32 [==============================] - 3s 2ms/step - loss: 2.4366 - accuracy: 0.2736
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 1.9687 - accuracy: 0.4249
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 1.7691 - accuracy: 0.4697
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 1.6460 - accuracy: 0.5045
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 1.5000 - accuracy: 0.5423
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 1.4075 - accuracy: 0.5692
Epoch 7/50
32/32 [==============================] - 0s 2ms/step - loss: 1.3735 - accuracy: 0.5990
Epoch 8/50
32/32 [==============================] - 0s 3ms/step - loss: 1.3179 - accuracy: 0.5871
Epoch 9/50
32/32 [==============================] - 0s 2ms/step - loss: 1.2777 - accuracy: 0.5970
Epoch 10/50
32/32 [==============================] - 0s 2ms/step - loss: 1.1719 - accuracy: 0.6358
Epoch 11/50
32/32 [

In [28]:
# 50 epochs
# New 
model.evaluate(test_dataset)

14/14 [==============================] - 0s 2ms/step - loss: 1.7091 - accuracy: 0.5476


[1.7091234922409058, 0.5475637912750244]

In [30]:
ans = model.predict(test_dataset)

In [31]:
print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1))))
print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1))),)

0.5475638051044084
              precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.50      0.43      0.46         7
           2       0.43      0.23      0.30        26
           3       0.71      0.38      0.50        13
           4       0.61      0.83      0.70        23
           5       1.00      0.50      0.67         2
           6       0.59      0.63      0.61        38
           7       0.55      0.56      0.55        41
           8       0.50      0.29      0.36         7
           9       0.40      0.50      0.44         4
          10       0.00      0.00      0.00         8
          11       0.67      0.71      0.69        84
          12       1.00      0.67      0.80        12
          13       0.75      0.48      0.58        44
          14       0.34      0.52      0.41        46
          15       0.38      0.45      0.41        40
          16       0.80      0.80      0.80        15
        

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
print('Building model...')
model_2 = Sequential()
model_2.add(Dense(2048, input_shape=(1024,)))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.2))
#model.add(Dense(1024, input_shape=(2048,)))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model_2.add(Dense(20))
model_2.add(Activation('softmax'))

model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#make one, takes in, double that amount, english and chichewa

Building model...


In [33]:
result = model_2.fit(train_dataset,
                    epochs=100, batch_size=32,
                    verbose=1)

Epoch 1/100
32/32 [==============================] - 0s 3ms/step - loss: 2.4622 - accuracy: 0.2388
Epoch 2/100
32/32 [==============================] - 0s 2ms/step - loss: 2.0347 - accuracy: 0.3781
Epoch 3/100
32/32 [==============================] - 0s 2ms/step - loss: 1.7830 - accuracy: 0.4716
Epoch 4/100
32/32 [==============================] - 0s 2ms/step - loss: 1.6455 - accuracy: 0.5045
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 1.5287 - accuracy: 0.5234
Epoch 6/100
32/32 [==============================] - 0s 2ms/step - loss: 1.4257 - accuracy: 0.5552
Epoch 7/100
32/32 [==============================] - 0s 2ms/step - loss: 1.3624 - accuracy: 0.5861
Epoch 8/100
32/32 [==============================] - 0s 2ms/step - loss: 1.2813 - accuracy: 0.5960
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 1.2652 - accuracy: 0.6030
Epoch 10/100
32/32 [==============================] - 0s 2ms/step - loss: 1.2001 - accuracy: 0.6299
Epoch 11/

In [34]:
# 100 epochs
model_2.evaluate(test_dataset)

14/14 [==============================] - 0s 2ms/step - loss: 2.3679 - accuracy: 0.5104


[2.3679163455963135, 0.5104408264160156]

In [35]:
ans = model_2.predict(test_dataset)

In [36]:
print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1))))
print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1))),)

0.5104408352668214
              precision    recall  f1-score   support

           0       0.33      0.50      0.40         2
           1       0.50      0.14      0.22         7
           2       0.40      0.15      0.22        26
           3       0.57      0.31      0.40        13
           4       0.67      0.78      0.72        23
           5       1.00      0.50      0.67         2
           6       0.68      0.50      0.58        38
           7       0.43      0.63      0.51        41
           8       0.50      0.29      0.36         7
           9       0.40      0.50      0.44         4
          10       0.00      0.00      0.00         8
          11       0.72      0.62      0.67        84
          12       1.00      0.50      0.67        12
          13       0.57      0.55      0.56        44
          14       0.31      0.52      0.39        46
          15       0.36      0.45      0.40        40
          16       0.80      0.80      0.80        15
        

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
print('Building model...')
model_3 = Sequential()
model_3.add(Dense(2048, input_shape=(1024,)))
model_3.add(Activation('relu'))
model_3.add(Dropout(0.2))
#model.add(Dense(1024, input_shape=(2048,)))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model_3.add(Dense(20))
model_3.add(Activation('softmax'))

model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#make one, takes in, double that amount, english and chichewa

Building model...


In [41]:
result = model_3.fit(train_dataset,
                    epochs=100, batch_size=64,
                    verbose=1)

Epoch 1/100
32/32 [==============================] - 0s 3ms/step - loss: 2.4179 - accuracy: 0.2607
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 1.9981 - accuracy: 0.4259
Epoch 3/100
32/32 [==============================] - 0s 4ms/step - loss: 1.7569 - accuracy: 0.4896
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 1.6342 - accuracy: 0.5114
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 1.5268 - accuracy: 0.5224
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 1.4447 - accuracy: 0.5672
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 1.3496 - accuracy: 0.5821
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 1.3001 - accuracy: 0.5980
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 1.2478 - accuracy: 0.6169
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 1.2086 - accuracy: 0.6189
Epoch 11/

In [42]:
model_3.evaluate(test_dataset)

14/14 [==============================] - 0s 2ms/step - loss: 2.3081 - accuracy: 0.5197


[2.308051347732544, 0.5197215676307678]

https://austingwalters.com/classify-sentences-via-a-multilayer-perceptron-mlp/
https://github.com/holbertonschool/deep-learning/blob/master/Class%20%233/Text%20Classification%20MLP%20Reuters%20Dataset.ipynb
https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2


Hidden state (embeddings):
https://github.com/huggingface/transformers/issues/1950